In [107]:
import numpy as np
import pandas as pd
import random

In [108]:
chunks=pd.read_csv("hash.txt",header=None,chunksize=1000000)

In [109]:
class HashLogLog:
    def __init__(self):
        random.seed(42)
        self.m = 2 ** 26 # around 1/2 of the cardinality of the universe
        self.p = 10 ** 9 + 9 # big prime number > m
        self.a = random.randint(1, self.m)
        self.b = random.randint(0, self.m)
    
    def get_hash(self, x):
        x = int(x, 16)
        bin_hash = bin(((self.a * x + self.b) % self.p) % self.m)[2:]
        return bin_hash.zfill(26)
    
    def leading_zeroes(self, x, k=4):
        hashed = self.get_hash(x)
        root = hashed[:k]
        hashed_count = hashed[k:]
        leading_zeroes = 0
        i = 0
        while ((hashed_count[i] != '1') and (i < 21)):
            leading_zeroes += 1
            i += 1
        return (root, leading_zeroes)

In [110]:
dict_of_buckets={}
for i in range (16):
    z=str(bin(i)[2:].zfill(4))
    dict_of_buckets["bucket"+z]=0
dict_of_buckets

{'bucket0000': 0,
 'bucket0001': 0,
 'bucket0010': 0,
 'bucket0011': 0,
 'bucket0100': 0,
 'bucket0101': 0,
 'bucket0110': 0,
 'bucket0111': 0,
 'bucket1000': 0,
 'bucket1001': 0,
 'bucket1010': 0,
 'bucket1011': 0,
 'bucket1100': 0,
 'bucket1101': 0,
 'bucket1110': 0,
 'bucket1111': 0}

In [111]:
my_hash = HashLogLog()

In [112]:
for chunk in chunks:
    binary = chunk[0].apply(my_hash.leading_zeroes)
    for i in binary:
        root=str(i[0])
        current_max=int(dict_of_buckets["bucket"+root])
        if int(i[1]) > current_max:
            dict_of_buckets["bucket"+root]=int(i[1])

In [113]:
dict_of_buckets

{'bucket0000': 21,
 'bucket0001': 21,
 'bucket0010': 21,
 'bucket0011': 21,
 'bucket0100': 21,
 'bucket0101': 21,
 'bucket0110': 21,
 'bucket0111': 21,
 'bucket1000': 21,
 'bucket1001': 21,
 'bucket1010': 21,
 'bucket1011': 21,
 'bucket1100': 21,
 'bucket1101': 21,
 'bucket1110': 21,
 'bucket1111': 21}

In [114]:
list_of_values=list()
for i in dict_of_buckets:
    list_of_values.append(dict_of_buckets[i])
n=np.mean(list_of_values)
n

21.0

In [116]:
estimated_cardinality=2**n
estimated_cardinality

2097152.0